# EWX Error Id

In [1]:
import os
import inspect
import time
import math
import json
import logging
import pickle
#import EWX_forecast as ewx

from energyworx_client.client import EWX
from energyworx_client import storage

import pandas as pd
import numpy as np
from scipy import stats
import pprint
import datetime as dt

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

from IPython.display import display

base = os.getcwd()
namespace = 'na.engie.com'
api = EWX(namespace)
pp = pprint.PrettyPrinter(indent = 1)

## Functions

In [92]:
def mse(a, b):
    return((sum(a - b))**2)

def meter_search(tag):
    file_result = api.search_files(tags = tag)['items']
    file_df = pd.DataFrame(file_result)
    
    ingest = file_df[file_df.readOnly == True]
    response = file_df[file_df.readOnly == False]
    
    ingest.reset_index(drop = True, inplace = True)
    response.reset_index(drop = True, inplace = True)
    
    return(file_df, ingest, response)

def download_response(blobkey):
    json_str = api.download_file(blob_key = blobkey)
    parse = json.loads(json_str)
    
    acct = parse['account']
    data = acct['timeseriesdataidr'][0]
    forecast = data['reads']
    
    ch3 = pd.DataFrame(forecast)
    ch3.columns = [str(c) for c in ch3.columns]
    ch3['t'] = pd.to_datetime(ch3['t'])
    ch3['v'] = pd.to_numeric(ch3['v'])
    
    return(ch3)

def download_ingest(blobkey):
    json_str = api.download_file(blob_key = blobkey)
    parse = json.loads(json_str)
    
    acct = parse['account']
    data = acct['timeseriesdataidr'][0]
    ingest = data['reads']
    
    idr = pd.DataFrame(ingest)
    idr.columns = [str(c) for c in idr.columns]
    idr['t'] = pd.to_datetime(idr['t'])
    idr['v'] = pd.to_numeric(idr['v'])
    
    return(idr)

def fix_ewx_bs(ch3, idr):

    #get beginning of idr
    jan_ingest = [(a.year == 2019 and a.month == 1 and a.day < 8) for a in idr.t]
    jan_idr = idr[jan_ingest]
    
    #shift ch 3 up six hrs
    time_diff = dt.timedelta(hours = 6)
    new_ch3_time = [time + time_diff for time in ch3['t']]
    ch3['t'] = new_ch3_time
    
    #group jan idr into hourly data
    beg = jan_idr[:12]
    beg.reset_index(drop = True, inplace = True)
    beg['h'] = [time.hour for time in beg['t']]
    beg['m'] = [time.minute for time in beg['t']]

    hourly = beg.groupby(beg['h']).sum()
    
    true_hr = beg[beg.m == 0]

    #make final jan idr hourly 
    real_val = hourly['v']
    real_time = true_hr['t']

    real_val.reset_index(drop = True, inplace = True)
    real_time.reset_index(drop = True, inplace = True)

    new_beg = pd.concat([real_time, real_val], axis = 1)
    
    #add to bad forecast
    final = pd.concat([new_beg, ch3], axis = 0)
    final.reset_index(inplace = True, drop = True)
    
    curr_yr = [a.year == 2019 for a in final['t']]
    final = final[curr_yr]
    final['t'] = pd.to_datetime(final['t'])
    final['v'] = pd.to_numeric(final['v'])
    
    return(final)

In [96]:
def mse_main(meter):

    file_df, ingest, response = meter_search(meter)
    ch3 = download_response(response.blobKey[0])
    idr = download_ingest(ingest.blobKey[0])
    fixed = fix_ewx_bs(ch3, idr)
    
    jan_response = [(a.year == 2019 and a.month == 1 and a.day < 8) for a in ch3.t]
    jan_ch3 = ch3[jan_response]
    jan_ch3.reset_index(drop = True, inplace = True)
    jan_ch3 = jan_ch3[:160]

    jan_fix = [(a.year == 2019 and a.month == 1 and a.day < 8) for a in fixed.t]
    fixed_jan = fixed[jan_fix]
    fixed_jan.reset_index(drop = True, inplace = True)
    fixed_jan = fixed_jan[:160]

    jan_ingest = [(a.year == 2019 and a.month == 1 and a.day < 8) for a in idr.t]
    jan_idr = idr[jan_ingest]
    jan_idr.reset_index(drop = True, inplace = True)
    jan_idr = jan_idr[:160]
    
    ch3_mse = mse(jan_ch3['v'], jan_idr['v'])
    fix_mse = mse(fixed_jan['v'], jan_idr['v'])
    
    return([meter, ch3_mse, fix_mse])

# 30 Days List

In [97]:
thirty = storage.read_csv('Last30DaysIDRVEE_20190716.csv')
thirty = thirty[['METERID', 'COMPLETEDDATE']]
thirty['COMPLETEDDATE'] = pd.to_datetime(thirty['COMPLETEDDATE'])
print thirty.head()

                               METERID COMPLETEDDATE
0                NEPOOL_NRI_8981705004    2019-06-16
1                   PJM_PPL_7990127017    2019-06-16
2                   PJM_PPL_9160141541    2019-06-16
3  MISO_AMERENCIPS_0745009519_32070513    2019-06-16
4                   PJM_PPL_7037124009    2019-06-16


In [98]:
uniq_date = np.unique(thirty['COMPLETEDDATE'])
uniq_date = uniq_date[::-1]

In [108]:
uniq_date = pd.to_datetime(uniq_date)
uniq_date

DatetimeIndex(['2019-07-16', '2019-07-15', '2019-07-14', '2019-07-13',
               '2019-07-12', '2019-07-11', '2019-07-10', '2019-07-09',
               '2019-07-08', '2019-07-07', '2019-07-06', '2019-07-05',
               '2019-07-04', '2019-07-03', '2019-07-02', '2019-07-01',
               '2019-06-30', '2019-06-29', '2019-06-28', '2019-06-27',
               '2019-06-26', '2019-06-25', '2019-06-24', '2019-06-23',
               '2019-06-22', '2019-06-21', '2019-06-20', '2019-06-19',
               '2019-06-18', '2019-06-17', '2019-06-16'],
              dtype='datetime64[ns]', freq=None)

In [127]:
new = uniq_date[4:]
new

DatetimeIndex(['2019-07-12', '2019-07-11', '2019-07-10', '2019-07-09',
               '2019-07-08', '2019-07-07', '2019-07-06', '2019-07-05',
               '2019-07-04', '2019-07-03', '2019-07-02', '2019-07-01',
               '2019-06-30', '2019-06-29', '2019-06-28', '2019-06-27',
               '2019-06-26', '2019-06-25', '2019-06-24', '2019-06-23',
               '2019-06-22', '2019-06-21', '2019-06-20', '2019-06-19',
               '2019-06-18', '2019-06-17', '2019-06-16'],
              dtype='datetime64[ns]', freq=None)

In [ ]:
for d in new:
    try:
        sub = thirty.loc[thirty.COMPLETEDDATE == d]
        sub.reset_index(drop = True, inplace = True)
        m = len(sub.METERID)
        print 'working with {} meters, expect {} minutes.'.format(m, round(m/15, 4))
        meter_data = []
    
        for i, meter in enumerate(sub.METERID):
            try:
                output = mse_main(meter)
                meter_data.append(output)
                if (i > 0 and i % 5 == 0):
                    print 'done with meter {} of {}.'.format(i, m)  
                
            except:
                print 'meter error, meter {} index {}'.format(meter, i)
                
        meter_df = pd.DataFrame(meter_data)
        meter_df.columns = ['meter', 'ch3_mse', 'fix_mse']
        meter_df['min_mse'] = meter_df[['ch3_mse', 'fix_mse']].min(axis = 1)
        fname = "".join(['EWX_CHECK_', str(d.month), '_', str(d.day), '.csv'])
        print meter_df.head()
        print 'writing file {}'.format(fname)
        storage.to_csv(meter_df, fname)
        
    except:
        print('no meters found for date {}'.format(d))
    
        

working with 149 meters, expect 9.0 minutes.


/opt/conda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


done with meter 5 of 149.
done with meter 10 of 149.
done with meter 15 of 149.
done with meter 20 of 149.
done with meter 25 of 149.
done with meter 30 of 149.
done with meter 35 of 149.
done with meter 40 of 149.
done with meter 45 of 149.
done with meter 50 of 149.
done with meter 55 of 149.
done with meter 60 of 149.
meter error, meter NYISO_CONED_393061163100008 index 61
meter error, meter NEPOOL_PSNH_80019150384 index 65
meter error, meter NEPOOL_PSNH_80019150202 index 66
done with meter 70 of 149.
meter error, meter NEPOOL_NRI_6455484014 index 74
meter error, meter NEPOOL_NRI_0234353014 index 75
done with meter 80 of 149.
done with meter 85 of 149.
meter error, meter NEPOOL_NRI_0234512013 index 86
meter error, meter NEPOOL_NRI_6455393018 index 90
meter error, meter NEPOOL_NRI_6455888012 index 94
done with meter 95 of 149.
done with meter 100 of 149.
done with meter 105 of 149.
meter error, meter PJM_COMED_3635043312 index 109
done with meter 110 of 149.
done with meter 115 of 14

done with meter 55 of 238.
done with meter 60 of 238.
done with meter 65 of 238.
done with meter 70 of 238.
meter error, meter NEPOOL_CLP_51495553075_329932006 index 73
done with meter 75 of 238.
done with meter 80 of 238.
done with meter 85 of 238.
done with meter 90 of 238.
done with meter 95 of 238.
done with meter 100 of 238.
done with meter 105 of 238.
done with meter 110 of 238.
done with meter 115 of 238.
done with meter 120 of 238.
meter error, meter NEPOOL_CLP_51495553075_329932006 index 121
meter error, meter PJM_PPL_7517353004 index 124
done with meter 125 of 238.
meter error, meter PJM_METED_08021075320002447038 index 130
meter error, meter ERCOT_CPT_1008901023815953730105 index 134
done with meter 135 of 238.
done with meter 140 of 238.
meter error, meter ERCOT_CPT_1008901023901067920115 index 143
meter error, meter ERCOT_CPT_1008901023900155230108 index 145
meter error, meter ERCOT_TNMP_10400513860890001 index 147
meter error, meter ERCOT_CPT_1008901023816338530105 index 

done with meter 75 of 172.
meter error, meter ERCOT_CPT_1008901001900108530107 index 80
meter error, meter ERCOT_CPT_1008901001900506680111 index 82
done with meter 85 of 172.
done with meter 90 of 172.
done with meter 95 of 172.
meter error, meter PJM_COMED_0534289006 index 96
done with meter 100 of 172.
done with meter 105 of 172.
done with meter 110 of 172.
done with meter 115 of 172.
done with meter 120 of 172.
done with meter 125 of 172.
meter error, meter PJM_DEOK_5620208503 index 127
done with meter 130 of 172.
done with meter 135 of 172.
meter error, meter PJM_PSEG_PE000009140834429191 index 137
done with meter 140 of 172.
done with meter 145 of 172.
done with meter 150 of 172.
done with meter 155 of 172.
done with meter 160 of 172.
done with meter 165 of 172.
done with meter 170 of 172.
                   meter       ch3_mse       fix_mse       min_mse
0     PJM_DQE_1443100321  1.172889e+04  4.499914e+06  1.172889e+04
1  NYISO_NIMO_9641418009  0.000000e+00  3.481000e+03  0.000

done with meter 160 of 214.
meter error, meter PJM_COMED_3570136045 index 161
done with meter 165 of 214.
meter error, meter PJM_PEPCODC_0550209821997001837943 index 166
done with meter 170 of 214.
meter error, meter PJM_APS_08043869565000613467 index 172
meter error, meter ERCOT_CPT_1008901011901336620117 index 174
done with meter 175 of 214.
meter error, meter ERCOT_CPT_1008901023816382550105 index 176
meter error, meter ERCOT_CPT_1008901023803333770100 index 177
meter error, meter ERCOT_CPT_1008901023802754420100 index 178
meter error, meter ERCOT_CPT_1008901023809862030100 index 179
meter error, meter ERCOT_CPT_1008901011900225860108 index 180
meter error, meter ERCOT_CPT_1008901006121093540100 index 181
meter error, meter ERCOT_CPT_1008901023803370220100 index 182
meter error, meter ERCOT_CPT_1008901023803426050100 index 183
meter error, meter ERCOT_CPT_1008901023808545620100 index 184
meter error, meter ERCOT_CPT_1008901023808545600100 index 185
meter error, meter ERCOT_CPT_10089

meter error, meter PJM_PSEG_PE000011231398667735 index 69
meter error, meter PJM_PSEG_PE000011255882067735 index 70
meter error, meter PJM_PSEG_PE000010011802567735 index 71
meter error, meter PJM_DPLDE_0550042740357002005400 index 72
done with meter 75 of 138.
meter error, meter NEPOOL_MECO_0091695046 index 76
done with meter 80 of 138.
meter error, meter PJM_COMED_0795181001 index 82
done with meter 85 of 138.
done with meter 90 of 138.
done with meter 95 of 138.
done with meter 100 of 138.
done with meter 105 of 138.
done with meter 110 of 138.
meter error, meter NEPOOL_NRI_5095713013 index 112
meter error, meter NEPOOL_NRI_7519476000 index 114
done with meter 115 of 138.
meter error, meter PJM_METED_08010187460002285812 index 120
done with meter 125 of 138.
meter error, meter PJM_COMED_1094228011 index 129
meter error, meter PJM_COMED_6184428097 index 130
done with meter 135 of 138.
                meter       ch3_mse       fix_mse       min_mse
0  PJM_PPL_3885485006  2.596726e+07 

done with meter 5 of 44.
done with meter 10 of 44.
done with meter 15 of 44.
done with meter 20 of 44.
done with meter 25 of 44.
done with meter 30 of 44.
done with meter 35 of 44.
done with meter 40 of 44.
                meter       ch3_mse       fix_mse       min_mse
0  PJM_PPL_1637039010  6.429998e+06  6.476496e+06  6.429998e+06
1  PJM_PPL_4266034015  1.420268e+04  1.537828e+04  1.420268e+04
2  PJM_PPL_4806034019  2.716875e+03  2.945086e+03  2.716875e+03
3  PJM_PPL_0037040010  1.718345e+10  1.660241e+10  1.660241e+10
4  PJM_PPL_4403065041  1.222740e+07  1.197599e+07  1.197599e+07
writing file EWX_CHECK_6_22.csv
working with 125 meters, expect 8.0 minutes.
meter error, meter PJM_PEPCO_0550165161837001586305 index 1
meter error, meter ERCOT_CPT_1008901023805435080100 index 3
done with meter 5 of 125.
meter error, meter NYISO_CONED_494021013000004 index 10
meter error, meter NYISO_CONED_494121209500005 index 11
meter error, meter NYISO_CONED_494051040500031 index 12
meter error, meter

meter error, meter ERCOT_ONCOR_10443720000247451 index 12
meter error, meter ERCOT_ONCOR_10443720004353702 index 13
meter error, meter ERCOT_ONCOR_10443720008785755 index 14
meter error, meter ERCOT_ONCOR_10443720009092021 index 15
meter error, meter ERCOT_ONCOR_10443720009216996 index 16
meter error, meter ERCOT_ONCOR_10443720004107748 index 17
meter error, meter ERCOT_ONCOR_10443720006995243 index 18
meter error, meter ERCOT_ONCOR_10443720008650130 index 19
done with meter 20 of 457.
meter error, meter ERCOT_ONCOR_10443720008232112 index 21
meter error, meter ERCOT_ONCOR_10443720004252704 index 22
meter error, meter ERCOT_ONCOR_10443720008921428 index 23
meter error, meter ERCOT_ONCOR_10443720006400089 index 24
done with meter 25 of 457.
meter error, meter ERCOT_ONCOR_10443720000247420 index 26
meter error, meter ERCOT_ONCOR_10443720004252921 index 27
meter error, meter ERCOT_ONCOR_10443720004353764 index 28
meter error, meter ERCOT_ONCOR_10443720007015997 index 30
meter error, meter